# MI-PAA, Úloha 1: Řešení problému batohu metodou hrubé síly a jednoduchou heuristikou

**Marián Hlaváč**, 18 Oct 2017 (hlavam30)  
marian.hlavac@fit.cvut.cz  
https://github.com/mmajko/knapsack-problem

## Zadání úlohy

- Naprogramujte řešení problému batohu hrubou silou (tj. exaktně). Na zkušebních datech pozorujte závislost výpočetního času na n.
- Naprogramujte řešení problému batohu heuristikou podle poměru cena/váha. Pozorujte
  závislost výpočetního času na n, průměrnou a maximální relativní chybu (tj. zhoršení proti exaktní metodě) v závislosti na n.


## Možné varianty řešení

Problém batohu je možné řešit hrubou silou, heuristicky, dynamickým programováním, algoritmem "meet-in-the-middle" a dalšími způsoby. Heuristická metoda řešení se dále dělí i podle zvolené heuristiky, např. jednoduchá greedy heuristika upřednostňuje nejdražší předměty. Takových heuristik existuje více, liší se rychlostí i komplexitou.

Zvolenou variantou pro první úlohu jsou řešení hrubou silou a jednoduchou heuristikou využívající poměr cena/váha u jednotlivých předmětů.

## Popis postupu řešení

Algoritmus a celý program poskytující výsledky je napsán v jazyce *Rust*. Tento program načte instance z předpřipravených datových souborů určených pro tuto úlohu a vypočte řešení hrubou silou a řešení za pomoci heuristiky. Zapíše délku provádění výpočtu a všechna data poskytne v CSV formátu.

Druhým nástrojem je pak *Jupyter Notebook*, ve kterém se poskytnutá data zpracují a vizualizují, zapíšou se výsledky těchto měření výpočtů a sepíše se zpráva.

### Kostra algoritmu

Kompletně celý algoritmus je k nahlédnutí ve zdrojových souborech programu. Pro rychlou představu je níže uveden krátký náhled na algoritmus výpočtu za pomoci heuristiky v jazyce Rust, který je aktuálně použit pro výsledky uvedené níže.

```rust
...
fn solve_heuristic(knap: &Knapsack) -> (u16, u16, u32) {
    let mut items: Vec<(usize, &KnapItem)> = knap.items.iter().enumerate().collect();
    items.sort_unstable_by(|a, b| (a.1.price / a.1.weight).cmp(&(b.1.price / b.1.weight)));
    
    let mut result_items: Vec<&KnapItem> = vec![];
    let mut total_weight = 0;
    for item in items {
        if item.1.weight + total_weight <= knap.capacity {
            result_items.push(item.1);
            total_weight += item.1.weight;
        } else {
            break;
        }
    }
...
```

Při výpočtu hrubou silou jsou pro každou jednotlivou instanci vyzkoušeny všechny kombinace umístění předmětů do batohu a následně je vybrána ta nejlepší vhodná (optimální). U této metody si můžeme být jisti, za předpokladu, že je výpočet kompletní, že jsme nalezli optimální řešení. 

Implementační detaily řešení lze nalézt ve zdrojových kódech. Byla použita bitová maska přítomnosti předmětu v batohu.

Výpočet heuristikou pak spočívá v seřazení pole předmětů podle kritéria heuristiky. Z tohoto pole jsou pak vybírány předměty do vyčerpání jeho kapacity.

## Surová naměřená (raw) data

Níže uvedená tabulka je náhled na kompletní surová výstupní data z programu. Data můžete sami (např. pro kontrolu) získat jednoduchým způsobem - spuštěním skriptu `generate.sh`, který vytvoří soubor `results.csv` obsahující tato data.

### Sloupce

Názvy sloupců se vyskytují i dále v textu, zde je jejich stručný popis:

- **knap_id** - identifikátor instance
- **item_count** - počet předmětů (konfigurace instance)
- **capacity** - kapacita batohu
- **method** - metoda výpočtu
  - *Bruteforce* je výpočet hrubou silou, *Heuristic* je heuristický výpočet (heuristika poměru váha/cena)
- **price** - vypočtená celková cena batohu
- **weight** - vypočtená celková váha batohu
- **bitmask** - bitmaska (jednoznačný identifikátor, maska přítomnosti předmětu) řešení
- **elapsed_ms** - doba výpočtu v milisekundách
- **optimal_price** - optimální cena batohu

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# Konfigurace vizualizace dat
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 5.0)
matplotlib.style.use('ggplot')
pd.options.display.max_rows = 10

# Čtení dat z běhu programu
raw = pd.read_csv("results.csv")
solutions = pd.read_csv("solutions.csv")

data = pd.merge(left=raw, right=solutions, on='knap_id').drop('item_count_y', axis=1)
data = data.rename(columns={
                'price_x': 'price', 
                'price_y': 'optimal_price', 
                'item_count_x': 'item_count'
            })

data

## Výsledky měření

Níže jsou uvedeny výsledky derivované z dat. Rychlosti řešení jsou seskupeny podle počtu předmětů a zprůměrovány. Výsledné průměrné časy jsou ke každé metodě řešení uvedeny jak tabulkou, tak grafem.

Měřítko grafů je lineární na vertikální ose.

### Rychlost řešení hrubou silou

Měření hrubou silou je výpočetně náročná metoda. Složitost je `O(2^n)`, takže lze očekávat rapidně vzrůstající trend doby nutné pro dokončení výpočtu touto metodou.

In [ ]:
# Výpis dat pro bruteforce metodu
bruteforce = data[data['method'] == 'Bruteforce']
mean_bruteforce = bruteforce.groupby('item_count').mean()[['elapsed_ms']]

mean_bruteforce

In [ ]:
# Vykreslení grafu dat pro bruteforce metodu
mean_bruteforce.plot(
    title = 'Závislost výpočetního času hrubou silou na n (počtu předmětů)',
    kind = 'bar',
)

Na výsledných datech lze pozorovat velmi rychle vzrůstající časovou závislost na počtu předmětů.

Díky složitosti `O(2^n)` obecně platí, že přidání jednoho dalšího předmětů zdvojnásobí celý výpočetní čas. Na výsledných datech lze tuto vlastnost snadno pozorovat.

Kupříkladu pro 20 předmětů byla průměrná doba výpočtu `266 ms`. Pro 22 předmětů `1100 ms`. To je 4.13 násobek původního čísla, šlo o rozdíl dvou předmětů, tudíž dvojnásobek času za každý přidaný předmět skutečně odpovídá.

### Rychlost řešení jednoduchou heuristikou

Rychlost řešení heuristikou by měla být podstatně méně časově náročná, než výpočet hrubou silou.

Jelikož je v průběhu řešení řazeno pole podle heuristiky, lze očekávat, že se vzrůstajícím počtem předmětů v poli se prodlouží i doba řazení tohoto pole.

Řazení pole má na starosti funkce jazyka *Rust* `Vec::sort_unstable_by()`, která slibuje složitost O(n log n) a je založena na [pattern-defeating quicksortu](https://github.com/orlp/pdqsort).

Zbytek algoritmu po seřazení je lineární (složitost v této části nezávisí na **n**, tedy počtu předmětů, ale na kapacitě batohu a velikosti předmětů - irelevantní pro naše pozorování).

In [ ]:
# Výpis dat pro heuristickou metodu
heuristic = data[data['method'] == 'Heuristic']
mean_heuristic = heuristic.groupby('item_count').mean()[['elapsed_ms']]

mean_heuristic

In [ ]:
# Vykreslení grafu pro heuristickou metodu
mean_heuristic.cumsum().plot(
    title = 'Závislost výpočetního času heuristikou na n (počtu předmětů)',
    kind = 'bar',
)

Na grafu lze pozorovat lineární vzrůst výpočetního času při vzrůstu počtu předmětů. Rozdíl je však v relativně nepatrný. Na grafu je důležité si všimnout rozsahu vertikální osy, která se se pohybuje v desetinách milisekund. Příčina pozorovaného vzrůstu je pak dána nutností řadit delší pole předmětů, což výpočetní čas ovlivňuje relativně minimálně.

Pokud bychom následovali lineární trend, citelnou časovou prodlevu bychom mohli pozorovat už při např. 100 000 předmětech, které by způsobily přibližně `133 ms` dlouhou prodlevu, za předpokladu, že takovou prodlevu předpokládáme za citelnou (tvrzení je spíše subjektivní záležitostí, nelze jednoznačně říct, co je dlouhá prodleva).

### Relativní chyby při výpočtu heuristikou

Jednoduchá heuristika, jako ta, která byla použita v této úloze, s největší pravděpodobností nebude schopná určit optimální řešení v každé instanci.

Relativní chybou lze určit úspěšnost heuristického výpočtu vůči exaktního výpočtu hrubou silou. Relativní chyby jsou uvedeny v procentech.

In [ ]:
pd.options.mode.chained_assignment = None

# Výpočet absolutních a relativních chyb u každého výpočtu heuristickou metodou
heuristic_data = data[data['method'] == 'Heuristic']
heuristic_data['error'] = heuristic_data['optimal_price'] - heuristic_data['price']
heuristic_data['relative_error_%'] = (heuristic_data['error'] / heuristic_data['optimal_price']) * 100

heuristic_data[['knap_id', 'item_count', 'optimal_price', 'price', 'error', 'relative_error_%']]

In [ ]:
# Zpracování relativních chyb pro každý počet předmětů jednotlivě
mean_vals = {}
rows_cnt = {}
max_vals = {}

for index, row in heuristic_data.iterrows():
    out_id = row['item_count']
    error = row['relative_error_%']
    
    mean_vals[out_id] = mean_vals[out_id] + error if out_id in mean_vals else error
    rows_cnt[out_id] = rows_cnt[out_id] + 1 if out_id in rows_cnt else 1
    if out_id in max_vals:
        max_vals[out_id] = error if error > max_vals[out_id] else max_vals[out_id]
    else:
        max_vals[out_id] = error
    
for index, row in mean_vals.items():
    mean_vals[index] = row / rows_cnt[index]

relative_errors = pd.DataFrame({'avg_rel_error_%': mean_vals, 'max_rel_error_%': max_vals})

# Výpis průměru a maxima relativní chyby pro každý počet předmětů
relative_errors

In [ ]:
relative_errors.plot(
    title = 'Relativní chyba v procentech v závislosti na n',
    kind = 'bar',
)

In [ ]:
# Statistiky pro všechny počty předmětů dohromady
relative_errors.describe()[1:][['avg_rel_error_%']]

Na grafu lze pozorovat mírně vzrůstající trend průměrné relativní chyby při vzrůstajícím počtu předmětů. O lineární vzrůst však s největší pravděpodobností nepůjde a dalo by se spíše předpokládat, že hodnota průměrné chyby se limitně blíží k 50%. Lepší odhad by poskytl větší vzorek dat, pro větší počty předmětů.

Grafická reprezentace maximální relativní chyby nenese žádnou podstatnou informaci (z grafu nelze vyčíst nic použitelného pro závěr).

## Závěr

Prvotní předpoklad, že výpočet problému batohu pomocí jednoduché heuristiky bude řádově rychlejší, než výpočet hrubou silou, se potvrdil. Na datech lze vidět důsledky složitosti algoritmu `O(2^n)`.

Průměrná relativní chyba při řešení pomocí heuristiky se ukázala, že je spíše vyšší a tak se podstatně liší i kvalita řešení obou metod. Metoda řešení pomocí heuristiky totiž vrací spíše méně kvalitní řešení (často `<50%`).

Lze tvrdit, že výpočet heuristikou se vyplácí až v bodě, kdy relativní chyba řešení nevzrůstá, protože víme, že časová náročnost řešení hrubou silou bude vzrůstat zaručeně vždy. K potvrzení tohoto tvrzení by byly vhodné další vzorky dat pro vyšší počty předmětů, aby mohl být trend relativní chyby jednoznačně určitelný. Výpočet dalších vzorků dat je však časově náročný a přesahuje hranice této úlohy.

Zdrojové soubory úlohy lze najít na GitHubu. Link je uveden v hlavičce zprávy.